In [0]:
storage_account_name = "storage4sushil"
storage_account_key = "gUfq3Q3auiB5zfLeNGPH1N6mxMJ0usXx3AeesaHsVgi6Is35YSuNt/l6MJeTAcoPD8StRVqvRJa6+AStbCUKlA=="

mount_points = {
    "raw-data": "/mnt/airline/raw-data",

}

containers = {
    "raw-data": "raw-data",
}

def mount_container(container_name, mount_point):
    if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
        dbutils.fs.mount(
            source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
            mount_point=mount_point,
            extra_configs={f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
        )
        print(f"Mounted {container_name} at {mount_point}")
    else:
        print(f"{mount_point} is already mounted.")

for container, mount_point in mount_points.items():
    mount_container(containers[container], mount_point)

dbutils.fs.mounts()

/mnt/airline/raw-data is already mounted.


[MountInfo(mountPoint='/mnt/taxi/silver', source='wasbs://silver@storage4sushil.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/Volumes', source='UnityCatalogVolumes', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo(mountPoint='/mnt/taxi/bronze', source='wasbs://bronze@storage4sushil.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/Volume', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/mnt/taxi/gold', source='wasbs://gold@storage4sushil.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/mnt/airline/raw-data', source='wasbs:/

In [0]:
storage_account_name = "storage4sushil"
storage_account_key = "gUfq3Q3auiB5zfLeNGPH1N6mxMJ0usXx3AeesaHsVgi6Is35YSuNt/l6MJeTAcoPD8StRVqvRJa6+AStbCUKlA=="

mount_points = {
    "processed-data": "/mnt/airline/processed-data",

}

containers = {
    "processed-data": "processed-data",
}

def mount_container(container_name, mount_point):
    if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
        dbutils.fs.mount(
            source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
            mount_point=mount_point,
            extra_configs={f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
        )
        print(f"Mounted {container_name} at {mount_point}")
    else:
        print(f"{mount_point} is already mounted.")

for container, mount_point in mount_points.items():
    mount_container(containers[container], mount_point)

dbutils.fs.mounts()

/mnt/airline/processed-data is already mounted.


[MountInfo(mountPoint='/mnt/taxi/silver', source='wasbs://silver@storage4sushil.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/Volumes', source='UnityCatalogVolumes', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo(mountPoint='/mnt/taxi/bronze', source='wasbs://bronze@storage4sushil.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/Volume', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/mnt/taxi/gold', source='wasbs://gold@storage4sushil.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/mnt/airline/raw-data', source='wasbs:/

In [0]:

flight_activity_file_path = "/mnt/airline/raw-data/Customer Flight Activity.csv"
loyalty_history_file_path = "/mnt/airline/raw-data/Customer Loyalty History.csv"

flight_activity_df = spark.read.csv(flight_activity_file_path, header=True, inferSchema=True)
loyalty_history_df = spark.read.csv(loyalty_history_file_path, header=True, inferSchema=True)


print("Customer Flight Activity Data:")
flight_activity_df.show(5)

print("Customer Loyalty History Data:")
loyalty_history_df.show(5)


Customer Flight Activity Data:
+--------------+----+-----+-------------+--------+------------------+---------------+---------------------------+
|Loyalty Number|Year|Month|Total Flights|Distance|Points Accumulated|Points Redeemed|Dollar Cost Points Redeemed|
+--------------+----+-----+-------------+--------+------------------+---------------+---------------------------+
|        100590|2016|    6|         12.0| 15276.0|           22914.0|           NULL|                       NULL|
|        100590|2010|    7|         12.0|  9168.0|           13752.0|           NULL|                       NULL|
|        100590|2010|    5|          4.0|  6504.0|            9756.0|           NULL|                       NULL|
|        100590|2011|   10|         NULL|    NULL|              NULL|          512.0|                       92.0|
|        100590|2015|    2|         NULL|    NULL|              NULL|           NULL|                       NULL|
+--------------+----+-----+-------------+--------+-------

In [0]:
from pyspark.sql.functions import col,sum
print("Null Counts in Customer Flight Activity Data:")
flight_nulls = flight_activity_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in flight_activity_df.columns])
flight_nulls.show()

flight_activity_df_cleaned = (
    flight_activity_df
    .na.fill({"Total Flights": 0, "Distance": 0, "Points Accumulated": 0, "Points Redeemed": 0, "Dollar Cost Points Redeemed": 0})
    .na.drop(subset=["Loyalty Number", "Year", "Month"]) 
)


Null Counts in Customer Flight Activity Data:
+--------------+----+-----+-------------+--------+------------------+---------------+---------------------------+
|Loyalty Number|Year|Month|Total Flights|Distance|Points Accumulated|Points Redeemed|Dollar Cost Points Redeemed|
+--------------+----+-----+-------------+--------+------------------+---------------+---------------------------+
|             0|   0|    0|       214122|  214122|            214122|         369051|                     369051|
+--------------+----+-----+-------------+--------+------------------+---------------+---------------------------+



In [0]:
print("Null Counts in Customer Loyalty History Data:")
loyalty_nulls = loyalty_history_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in loyalty_history_df.columns])
loyalty_nulls.show()

loyalty_history_df_cleaned = (
    loyalty_history_df
    .na.fill({"Salary": 0, "CLV": 0,"Cancellation Year": 0,"Cancellation Month": 0})
    .na.fill("Unknown", subset=["Education", "Marital Status", "Gender", "Province", "City", "Postal Code","Cancellation Year","Cancellation Month"])
    .na.drop(subset=["Loyalty Number"])  
)


Null Counts in Customer Loyalty History Data:
+--------------+-------+--------+----+-----------+------+---------+------+--------------+------------+---+---------------+---------------+----------------+-----------------+------------------+
|Loyalty Number|Country|Province|City|Postal Code|Gender|Education|Salary|Marital Status|Loyalty Card|CLV|Enrollment Type|Enrollment Year|Enrollment Month|Cancellation Year|Cancellation Month|
+--------------+-------+--------+----+-----------+------+---------+------+--------------+------------+---+---------------+---------------+----------------+-----------------+------------------+
|             0|      0|       0|   0|          0|     0|        0|  4238|             0|           0|  0|              0|              0|               0|            14670|             14670|
+--------------+-------+--------+----+-----------+------+---------+------+--------------+------------+---+---------------+---------------+----------------+-----------------+---------

In [0]:
from pyspark.sql.functions import col
dataframes = [loyalty_history_df_cleaned, flight_activity_df_cleaned]  
for df in dataframes:
    columns = df.columns  
    for column in columns:
        if dict(df.dtypes)[column] in ['int', 'double', 'float']:  
            negative_values = df.filter(col(column) < 0).count()  
            if negative_values > 0:
                print(f"'{column}'  Negative.")
            else:
                print(f"'{column}'  ALl Positive.")

'Loyalty Number'  ALl Positive.
'Salary'  Negative.
'CLV'  ALl Positive.
'Enrollment Year'  ALl Positive.
'Enrollment Month'  ALl Positive.
'Cancellation Year'  ALl Positive.
'Cancellation Month'  ALl Positive.
'Loyalty Number'  ALl Positive.
'Year'  ALl Positive.
'Month'  ALl Positive.
'Total Flights'  ALl Positive.
'Distance'  ALl Positive.
'Points Accumulated'  ALl Positive.
'Points Redeemed'  ALl Positive.
'Dollar Cost Points Redeemed'  ALl Positive.


In [0]:
from pyspark.sql.functions import when

loyalty_history_df_cleaned = loyalty_history_df_cleaned.withColumn(
    "Salary",
    when(col("Salary") < 0, 0).otherwise(col("Salary"))
)

dataframes = [loyalty_history_df_cleaned, flight_activity_df_cleaned]
for df in dataframes:
    columns = df.columns
    for column in columns:
        if dict(df.dtypes)[column] in ['int', 'double', 'float']:
            negative_values = df.filter(col(column) < 0).count()
            if negative_values > 0:
                print(f"'{column}'  Negative.")
            else:
                print(f"'{column}'  ALl Positive.")


'Loyalty Number'  ALl Positive.
'Salary'  ALl Positive.
'CLV'  ALl Positive.
'Enrollment Year'  ALl Positive.
'Enrollment Month'  ALl Positive.
'Cancellation Year'  ALl Positive.
'Cancellation Month'  ALl Positive.
'Loyalty Number'  ALl Positive.
'Year'  ALl Positive.
'Month'  ALl Positive.
'Total Flights'  ALl Positive.
'Distance'  ALl Positive.
'Points Accumulated'  ALl Positive.
'Points Redeemed'  ALl Positive.
'Dollar Cost Points Redeemed'  ALl Positive.


In [0]:
def check_nulls(df, name):
    print(f"Checking nulls in {name}:")
    null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
    null_counts.show()

check_nulls(loyalty_history_df_cleaned, "loyalty_history_df_cleaned")
check_nulls(flight_activity_df_cleaned, "flight_activity_df_cleaned")   

Checking nulls in loyalty_history_df_cleaned:
+--------------+-------+--------+----+-----------+------+---------+------+--------------+------------+---+---------------+---------------+----------------+-----------------+------------------+
|Loyalty Number|Country|Province|City|Postal Code|Gender|Education|Salary|Marital Status|Loyalty Card|CLV|Enrollment Type|Enrollment Year|Enrollment Month|Cancellation Year|Cancellation Month|
+--------------+-------+--------+----+-----------+------+---------+------+--------------+------------+---+---------------+---------------+----------------+-----------------+------------------+
|             0|      0|       0|   0|          0|     0|        0|     0|             0|           0|  0|              0|              0|               0|                0|                 0|
+--------------+-------+--------+----+-----------+------+---------+------+--------------+------------+---+---------------+---------------+----------------+-----------------+---------

In [0]:
loyalty_history_df_cleaned = loyalty_history_df_cleaned.dropDuplicates()
flight_activity_df_cleaned = flight_activity_df_cleaned.dropDuplicates()


print("Row counts after removing duplicates:")
print(f"Loyalty History: {loyalty_history_df_cleaned.count()}")
print(f"Flight Activity: {flight_activity_df_cleaned.count()}")


Row counts after removing duplicates:
Loyalty History: 16737
Flight Activity: 386873


In [0]:
loyalty_history_df_cleaned.printSchema()
flight_activity_df_cleaned.printSchema()

root
 |-- Loyalty Number: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Province: string (nullable = false)
 |-- City: string (nullable = false)
 |-- Postal Code: string (nullable = false)
 |-- Gender: string (nullable = false)
 |-- Education: string (nullable = false)
 |-- Salary: integer (nullable = false)
 |-- Marital Status: string (nullable = false)
 |-- Loyalty Card: string (nullable = true)
 |-- CLV: double (nullable = false)
 |-- Enrollment Type: string (nullable = true)
 |-- Enrollment Year: integer (nullable = true)
 |-- Enrollment Month: integer (nullable = true)
 |-- Cancellation Year: integer (nullable = false)
 |-- Cancellation Month: integer (nullable = false)

root
 |-- Loyalty Number: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Total Flights: double (nullable = false)
 |-- Distance: double (nullable = false)
 |-- Points Accumulated: double (nullable = false)
 |-- Points Redeemed: doub

In [0]:
loyalty_history_df_cleaned.write.mode("overwrite").parquet("/mnt/airline/processed-data/loyalty_history_df_cleaned")
flight_activity_df_cleaned.write.mode("overwrite").parquet("/mnt/airline/processed-data/flight_activity_df_cleaned")